<a href="https://colab.research.google.com/github/bonvech/MSU-AI/blob/main/Copy_of_EX10_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Вспомогательный код

Чтобы результаты экспериментов воспроизводились, зафиксируем seed's:

In [ ]:
import torch
import random
import numpy as np


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


set_random_seed(42)

Для выполнения задания рекомендуется использовать среду с аппаратным ускорителем GPU:

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Установим Transformers и необходимые библиотеки:

In [ ]:
!pip install -q -U transformers accelerate git+https://github.com/huggingface/peft.git

In [ ]:
!pip install -q datasets==2.16.1

In [ ]:
!pip install -q trl
!pip install -q bitsandbytes==0.41.3 einops #wndb
!pip install -q peft==0.6.2

# Задание 1. "Естественность" языка

Важная задача в NLP — оценка качества сгенерированного текста.
Одной из самых распространённых метрик является [перплексия ✏️[blog]](https://habr.com/ru/companies/wunderfund/articles/580230/). Это классический подход, и существует целый ряд идейно близких метрик.

Сейчас появляются метрики, которые не просто применяются "для нейросетей", они сами являются нейросетями. [RuCoLa ✏️[blog]](https://habr.com/ru/articles/667336/) — одна из характерных представителей. Это целый проект, со своим API и [лидербордом 🛠️[doc]](https://rucola-benchmark.com/leaderboard).

Подобные рейтинги, кстати, позволяют нам выбирать модели под наши задачи.

Это модель, обученная на тщательно отобранных лингвистами вручную предложениях, которая производит бинарную классификацию (*приемлемо* или *неприемлемо*), а также выдаёт оценку "уверенности" в своём ответе.

Задачи:

* Возьмите два предложенных текста по 12 предложений, получите для каждого предложения цифры по метрикам RuCoLa (не разделяя по классам) и перплексия.
* Аугментируйте предложения с помощью библиотек для аугментации текстов, посчитайте по ним метрики.
* Постройте "ящики с усами" по полученным цифрам для данных до и после аугментаци.
* Сравните визуально по паре предложений с самыми худшими и самыми лучшими значениями метрик.

Установка и импорт необходимых библиотек:

In [ ]:
!pip install -q nlpaug

In [ ]:
import torch
import numpy as np
import transformers
import matplotlib.pyplot as plt
from transformers import pipeline
from transformers import AutoModelForMaskedLM, AutoTokenizer

Посмотрим, как получить метрику RuCoLa. `LABEL_1` означает, что предложение пригодное, `LABEL_0` — непригодное, `score` — степень уверенности модели в этом в промежутке от 0 до 1.

    
> Это специально обученная модель для выдачи метрик, применение к другим типам задач может привести к произвольным результатам или не сработать вовсе.

In [ ]:
from warnings import simplefilter
from transformers import pipeline

simplefilter("ignore", UserWarning)

classifier = pipeline("text-classification", model="RussianNLP/ruRoBERTa-large-rucola")

Оценим текст, сгенерированный нейросетью.

In [ ]:
text_gen = [
    "Однажды вечером я решил приготовить борщ.",
    "Я купил все необходимое ингредиенты и начал готовить.",
    "Через несколько часов борщ готовился, и я решил приготовить блины и пельмени.",
    "Я начал приготавливать блины, используя те же ингредиенты, что и для борща.",
    "Я добавил муку, яйца, молоко, соль и перец, и смешал все вместе.",
    "Затем я выложил тесто в форму и выпекал блины.",
    "После того, как блины готовы, я начал приготавливать пельмени.",
    "Я добавил муку, яйца, молоко и соль, и смешал все вместе.",
    "Затем я выложил тесто в формы и выпекал пельмени.",
    "Когда борщ, блины и пельмени готовы, я подал все вместе на стол.",
    "Мы наслаждались вкусным борщем, блинами и пельменами.",
    "Это было настоящее удовольствие!",
]

In [ ]:
gen = classifier(text_gen)

Посчитаем среднее качество текста и уверенность в ответе.

In [ ]:
def average_metric(res):
    good = 0
    bad = 0
    av_score = 0
    for i in range(len(res)):
        if res[i]["label"] == "LABEL_1":
            good += 1
            av_score += res[i]["score"]
        else:
            bad += 1
            av_score += res[i]["score"]
    print("Good:", round(good / len(res), 2))
    print("Bad:", round(bad / len(res), 2))
    print("Mean Score:", round(av_score / len(res), 3))

In [ ]:
average_metric(gen)

Обратите внимание на разделение по классам. Вы, скорее всего, не являетесь профессиональным лингвистом, но, возможно, сможете оценить (не)естественность предложений.

Ниже, для примера, текст из учебника "Русский язык для иностранцев".

In [ ]:
text_real = [
    "Привет, любимая.",
    "Как дела?",
    "Что делаешь?",
    "Я сейчас в метро, еду на урок русского языка.",
    "Обычно я езжу на урок на метро.",
    "Метро - очень популярный вид транспорта в Москве, потому что в городе всегда пробки.",
    "Ты знаешь, что значит слово «пробка»?",
    "По-русски «пробка» - это когда на улице все машины стоят и не едут.",
    "А ещё пробка есть в бутылке.",
    "В метро всегда много народу и очень шумно.",
    "Но оно очень красивое, как музей.",
    "И ещё оно быстрое и удобное.",
]

In [ ]:
real = classifier(text_real)

In [ ]:
real

In [ ]:
average_metric(real)

Воспользуемся метрикой [перплексия ✏️[blog]](https://habr.com/ru/companies/wunderfund/articles/580230/).

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer


model_name = "cointegrated/rubert-tiny2"
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors="pt")
    repeat_input = tensor_input.repeat(tensor_input.size(-1) - 2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill(masked_input != tokenizer.mask_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())


print(
    score(
        sentence="London is the capital of Great Britain.",
        model=model,
        tokenizer=tokenizer,
    )
)
print(
    score(
        sentence="London is the capital of South America.",
        model=model,
        tokenizer=tokenizer,
    )
)

Посчитайте перплексию для текстов `text_gen` и `text_real`, а также средние значения метрик.

In [ ]:
perplex_text_gen = # Your code here
perplex_text_real = # Your code here

Возьмите библиотеку аугментации текстов `nlpaug` из прошлой лекции и попортите оба текста. Посчитайте по ним перплексию, RuCoLa и средние значения RuCoLa функцией выше.

Сначала сделайте аугментацию сгенерированного текста.

In [ ]:
import nlpaug.augmenter.word as naw


augment = naw.ContextualWordEmbsAug(model_path="cointegrated/rubert-tiny2", action="substitute", aug_p = 0.2)
augmented_text_gen = # Your code here

augmented_text_gen

Посчитайте метрики.

In [ ]:
gen_aug = classifier(augmented_text_gen)
gen_aug

In [ ]:
# Your code here

In [ ]:
# Your code here

Сделайте аугментацию второго текста.

In [ ]:
# Your code here

Посчитайте метрики.

In [ ]:
# Your code here

Рассчитайте ящики с усами по обоим метрикам. Для это постройте два subplot'a, на каждом из них выведите по 4 ящика. На левом subplot'е — RuCoLa, на правом — перплексия.

In [ ]:
real_scores = # Your code here
gen_scores = # Your code here
real_aug_scores = # Your code here
gen_aug_scores = # Your code here

Ниже представлен код для удобства визуализации.

In [ ]:
import matplotlib.pyplot as plt


# Male subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

# Dict
data_r = {
    "gen_scores": gen_scores,
    "real_scores": real_scores,
    "gen_aug_scores": gen_aug_scores,
    "real_aug_scores": real_aug_scores,
}
data_p = {
    "text_gen": perplex_text_gen,
    "text_real": perplex_text_real,
    "text_gen_aug": perplex_text_gen_aug,
    "text_real_aug": perplex_text_real_aug,
}

axes[0].boxplot(data_r.values())
axes[0].set_xticklabels(data_r.keys())
axes[0].set_title("RuCoLa")


axes[1].boxplot(data_p.values())
axes[1].set_xticklabels(data_p.keys())
axes[1].set_title("Perplexity")

plt.ylabel("Values")

plt.tight_layout()
plt.show()

Можно ли делать вывод о качестве текстов (и, соответственно, моделей, которые их генерируют)?

**Дополнительно**

* Можно взять весь [датасет RuCoLa 🐾[git]](https://github.com/RussianNLP/RuCoLA) и проверить его согласно метрикам RuCoLa и перплексия, построить "ящики с усами".

Для скачивания датасета можно воспользоваться командой:


```
!git clone https://github.com/RussianNLP/RuCoLA.git data/data
```



## Формат результата

Получить "ящики с усами" метрик.

# Задание 2. Классификация с помощью BERT

Возьмите набор данных эмоциональных окрасок отзывов [emotions dataset 🛠️[doc]](https://huggingface.co/datasets/emotion). В датасете 5 классов. Получите эмбеддинг из BERT-подобного кодировщика. Классифицируйте тексты с помощью методов ML или  нейросети, используя эмбеддинги в качестве входов.

Произведите fine-tuning кодировщика на ваших данных (классификатор на основе BERT) и сравните, как изменилось качество классификации.

In [ ]:
!pip install -q datasets==2.16.1 evaluate

In [ ]:
import torch
import evaluate
import numpy as np
import pandas as pd
from datasets import Dataset
from datasets import load_dataset
from datasets.dataset_dict import DatasetDict
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

Взглянем на датасет.

In [ ]:
from warnings import simplefilter

simplefilter("ignore", UserWarning)

train = load_dataset("SetFit/emotion", split="train")
train_df = pd.DataFrame({"text": train["text"], "labels": train["label"]})
train_df.head()

Инициализация токенайзера и модели. Перед тем, как отправить текст в модель, его следует токенизировать.

**Учитывайте**, что:
* Мы посмотрели несколько способов получения эмбеддингов из моделей BERT. Воспользуйтесь одним из них.
* Длина эмбеддинга больше 200 в данной задаче не потребуется. Чем длиннее эмбеддинг, тем медленнее всё будет учиться.
* В каждой части задания нужно задать по отдельной модели.
* Библиотеки в первой и второй части задания могут конфликтовать. Используйте "перезапуск сеанса", если так случилось.

Возьмите модель `bert-tiny` и соответствующий токенизатор.

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(# Your code here)
model = AutoModel.from_pretrained(# Your code here)
model.to(device)

Посмотрим, как устроен датасет и какой у него баланс классов:

In [ ]:
# Your code here

Оставьте 5000 объектов, поделив их на обучение, валидацию, тест.

Если вы будете семплировать, в функцию `sample` из `pandas` можно подать веса, с которыми будет происходить семплирование.

Один из вариантов — создать столбец, в каждой строке которого будет необходимая частота семплирования, и именно этот объект подавать в качестве весов.

In [ ]:
# Your code here

In [ ]:
# Your code here

Также имеет смысл проверить, что в колонке `labels` данные типа `int`. Если нет, то привести к этому типу.

In [ ]:
# Your code here

In [ ]:
from sklearn.model_selection import train_test_split

# Your code here

Проверьте, какие объекты и в каком количестве представлены в метках в трёх выборках.

In [ ]:
# Your code here

Получите и сохраните векторные представления текстов, которые возвращает модель.

In [ ]:
train_tok = []
# Your code here

In [ ]:
val_tok = []
# Your code here

In [ ]:
test_tok = []
# Your code here

Возьмите ML-классификатор, обучите его и постройте `classification_report`.

Базово можно взять `RandomForestClassifier`.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Your code here

In [ ]:
from sklearn.metrics import classification_report

# Your code here

Здесь не требуется получить 100% Presicion и Recall. Однако проверьте, что вы верно разделили данные на 3 подвыборки и что значения метрик не находятся в районе нуля.

**Попробуйте дообучить BERT и произвести классификацию заново.**

In [ ]:
import evaluate

metric = evaluate.load("f1")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(
        predictions=predictions, references=labels, average="weighted"
    )

In [ ]:
from datasets import Dataset
from datasets.dataset_dict import DatasetDict

d = {
    "train": Dataset.from_dict({"label": y_train, "text": x_train}),
    "val": Dataset.from_dict({"label": y_val, "text": x_val}),
    "test": Dataset.from_dict({"label": y_test, "text": x_test}),
}

dataset = DatasetDict(d)

In [ ]:
dataset

Настройте токенизатор (если это не было сделано выше) так, чтобы производились дополнение (padding) и обрезка предложений. Имеет смысл указать длину получаемых векторных представлений (`model_max_length`).

In [ ]:
# Your code here

Напишите функцию, которая будет токенизировать ваш текст в `DatasetDict`.

In [ ]:
# Your code here

Токенизируйте ваши данные:

In [ ]:
# Your code here

Посмотрите, как  выглядит ваш `DatasetDict`, какие размеры (`shape`) сущностей, которые он содержит.

In [ ]:
# Your code here

In [ ]:
# Your code here

Импортируйте модель с учётом того, что вы уже решаете не просто задачу построения эмбеддингов, а задачу классификации.

In [ ]:
# Your code here
model_for_classification =

In [ ]:
# model

Задайте параметры, обучите модель:

In [ ]:
from transformers import Trainer, TrainingArguments

# Your code here
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=#,
    per_device_eval_batch_size=#,
    num_train_epochs= #,#about 1.5 minutes for 1 epoch
    weight_decay=#,
    evaluation_strategy=#,
    save_strategy= #,
    load_best_model_at_end= #,
    push_to_hub=False,
)



trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_data['train'],         # training dataset
    eval_dataset=tokenized_data['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Мы дообучили модель. Теперь проверим качество предсказания на тесте.

Можете использовать `trainer.predict`.

In [ ]:
# Your code here

In [ ]:
from sklearn.metrics import classification_report

# Your code here

Если у вас получилось 100%, проверьте, что с чем вы сравниваете. По умолчанию `predict` возвращает логиты и истинные метки, которые были изначально в переданном объекте.

**Дополнительно**

* Попробуйте классифицировать, использовав такие методы, как TF-IDF, word2vec и другие.

* Попробуйте использовать косинусную меру для оценки близости текстов. Получается ли с её помощью разделять классы?

## Формат результата

Получить значение качества классификации.

# Задание 3. Генерация текста

Возьмите произведение Гете "Фауст" и обучите на нем модель генерации текста.

При обучении игнорируйте знаки препинания и номера страниц.

Используйте предобученную модель `sberbank-ai/rugpt3small_based_on_gpt2` из библиотеки Hugging Face. Попробуйте разные параметры модели (например, разные режимы генерации). Как меняется результат?


Импорт необходимых библиотек:

In [ ]:
import torch
from transformers import Trainer, TrainingArguments
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling

## Загрузка и подготовка данных

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/Faust.txt

In [ ]:
with open("Faust.txt") as text_file:
    faust_text = "".join(text_file.readlines())

In [ ]:
faust_text

In [ ]:
len(faust_text.split(" "))

Удалим знаки управляющих символов и приведем текст к нижнему регистру:

In [ ]:
text = (
    faust_text.replace("\n", " ")
    .replace("\r", "")
    .replace("\ufeff", "")
    .replace("\x0c", "")
)
text = "".join([char for char in text if char.isalpha() or char == " "])
text = text.lower()

In [ ]:
text

Запишем обработанные данные в текстовый файл:

In [ ]:
with open(train_path, "w") as f:
    f.write(text)

## Предобученная sberbank-ai GPT2

Возьмите предобученную модель и попробуйте сгенерировать текст с ее помощью. Оцените результаты.

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel


model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)

input_ids = # Your code here

sample_output = model.generate(
# Your code here
)

generated_text = tokenizer.decode(sample_output[0], skip_special_tokens=True)

In [ ]:
print(generated_text)

## Файнтюнинг sberbank-ai GPT2

Дообучите модель. Для этого сначала создадим датасет и `data_collator`, который нарезает текст на оптимальные по длине куски:

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from warnings import simplefilter

simplefilter("ignore", category=FutureWarning)

train_path = "Faust.txt"

train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import Trainer, TrainingArguments

model.to(device)

# Your code here
training_args = TrainingArguments(
    output_dir=, #The output directory
    overwrite_output_dir=, #overwrite the content of the output directory
    num_train_epochs=, # number of training epochs
    per_device_train_batch_size=, # batch size for training
    per_device_eval_batch_size=,  # batch size for evaluation
    warmup_steps=,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=, # to make "virtual" batch size larger
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = # Your code here # Optimizer and lr scheduler
)

In [ ]:
trainer.train()

In [ ]:
# Probability beam sampling example
text = "Ходил"
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    out = model.generate(
        input_ids,
        do_sample=True,
        num_beams=2,
        temperature=1.5,
        top_p=0.9,
        max_length=200,
    )

generated_text = list(map(tokenizer.decode, out))[0]

In [ ]:
print(generated_text)

## Другая модель с HuggingFace

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel


tokenizer = GPT2Tokenizer.from_pretrained("cometrain/neurotitle-rugpt3-small")
model = GPT2LMHeadModel.from_pretrained("cometrain/neurotitle-rugpt3-small")

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

train_path = "Faust.txt"

train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import Trainer, TrainingArguments

model.to(device)

# Your code here
training_args = TrainingArguments(
    output_dir=, #The output directory
    overwrite_output_dir=, #overwrite the content of the output directory
    num_train_epochs=, # number of training epochs
    per_device_train_batch_size=, # batch size for training
    per_device_eval_batch_size=,  # batch size for evaluation
    warmup_steps=,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=, # to make "virtual" batch size larger
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = # Your code here # Optimizer and lr scheduler
)

In [ ]:
trainer.train()

In [ ]:
# Probability beam sampling example
text = "все"
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    out = model.generate(
        input_ids,
        do_sample=True,
        num_beams=1,
        temperature=1.6,
        top_p=0.7,
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,
    )

generated_text = list(map(tokenizer.decode, out))[0]

In [ ]:
print(generated_text)

## Формат результата

Сгенерированный текст

Пример текста:

"все все от бесстыдные старой

все в нем получше все стремленья

поддержки с собой в сердце воздух своей

и в вечной страсти восстанет свой предлог

привет вам слуга в сладком страшней стране

и в мире все вражда станет станет

в поле на пользу своим воспоминанья"


# Задание 4. QLoRa

Дообучите большую языковую модель.

В этом задании мы предлагаем для проверки подхода взять готовый датасет, загрузка которого реализована ниже.

Идея задания состоит в том, чтобы вы прошли весь путь по запуску дообучения по методу LoRa. Здесь не требуется брать большие модели или учить сотни эпох.

*Перед выполнением задания может потребоваться перезагрузить среду.*

В задании используется `SFTTrainer` [🛠️[doc]](https://huggingface.co/docs/trl/sft_trainer) (Supervised Fine Tune Trainer).

Это обёртка над классом `Trainer` для обучения моделей на наборах инструкций с использованием адаптеров PEFT.

Установка и импорт необходимых библиотек:

In [ ]:
!pip install -q zstandard jsonlines

In [ ]:
import bitsandbytes as bnb
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

## Загрузка и подготовка данных

С помощью поисковика найдём [небольшую модель 🛠️[doc]](https://llm.extractum.io/model/Defetya%2Fopenllama-3b-saiga,5NvQyABIZu8ZZa2NCI6BQN).

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_name = "Defetya/openllama-3b-saiga"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)
model.config.use_cache = False

Загрузите токенизатор. Вам может помочь параметр `trust_remote_code=True`.

In [ ]:
from transformers import AutoTokenizer

tokenizer =  # Your code here
tokenizer.pad_token =  # Your code here

Воспользуйтесь библиотекой peft и настройте `peft_config`, определите `target_modules` для LoRa:

In [ ]:
lora_alpha = # Your code here
lora_dropout = # Your code here
lora_r = # Your code here

In [ ]:
# Your code here

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=# Your code here
    target_modules = # Your code here
    lora_dropout=# Your code here
    r=# Your code here
    bias=# Your code here
    task_type="CAUSAL_LM"
)

Загрузите датасет:

In [ ]:
from datasets import load_dataset

dataset_name = "IlyaGusev/ru_turbo_alpaca"
dataset = load_dataset(dataset_name, split="train", trust_remote_code=True)

Датасет представляет собой набор простых вопросов и ответов.

In [ ]:
dataset

Проверьте, насколько качественно модель умеет работать. Задайте вопрос из только что загруженного датасета.

In [ ]:
# Your code here

Необходимо в одну инструкцию заложить и вопрос, и ответ. Преобразуйте датасет так, чтобы можно было подать единственное текстовое поле далее в `SFTrainer`.

In [ ]:
# Your code here

Настройте параметры и передайте их в `TrainingArguments`:

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = # Your code here
gradient_accumulation_steps = # Your code here
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = # Your code here
learning_rate = # Your code here
max_grad_norm = # Your code here
max_steps = # Your code here
warmup_ratio = # Your code here
lr_scheduler_type = # Your code here

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Подайте всё в `SFTTrainer`:

In [ ]:
from trl import SFTTrainer

max_seq_length = 256
tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,  # your dataset
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
trainer.train()

Изменилось ли качество генерации после обучения?
Попробуйте вновь подать тот же промпт. Модель должна выдать ответ, более похожий на уже виденный каноничный ответ.

In [ ]:
# Your code here

## Формат результата

Сгенерированный текст ответа на поданный вопрос. Пример:


```
Опишите, как сделать горшок из глины.

Горшок из глины
Глина (2 столовые ложки)

Инструкции:

1. Размешайте глину в ведре до однородности.

2. Вам понадобится деревянная палочка для насыпки глины.

3. Выложите горшок в целлофан и привяжите его к стенке горшка для дополнительной устойчивости.

4. Наполните горшок поочередно смесью, начиная отверхности.

5. Разложите насыпку из глины по стенкам горшка.

6. Возьмите лопату, кисточку и палочку, чтобы создать каркас горшка.

7. Украсьте горшок из глины различными текстурами и цветами, такими как камень, металл, стекло и другие материалы.

Посмотрите видео, чтобы узнать больше о том, как сделать горшок из глины:

Приятного творчества!
```